In [84]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
from sklearn.preprocessing import MinMaxScaler
import requests
from datetime import datetime as dt
import asyncio
from operator import attrgetter

import aiohttp
from prettytable import PrettyTable

from fpl import FPL
from fpl.utils import team_converter
from colorama import Fore, init
from datetime import date

In [2]:
session = requests.session()
url = 'https://users.premierleague.com/accounts/login/'
creds = ("knowingvivek@gmail.com", "Pl1nc#$$")
payload = {
    'password': creds[0],
    'login': creds[1],
    'redirect_uri': 'https://fantasy.premierleague.com/a/login',
    'app': 'plfpl-web'
}
session.post(url, data=payload)
global team_map

In [3]:
def get_json(file_path):
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    jsonResponse = r.json()
    with open(file_path, 'w') as outfile:
        json.dump(jsonResponse, outfile)
get_json('fpl.json')
with open('fpl.json') as json_data:
    d = json.load(json_data)
global team_map
team_map ={}
for rec in d["teams"]:
    name = rec["name"]
    i = rec["id"]
    team_map[i]=name
global pos_map
pos_map = {1: "Goalkeeper", 2: "Defender", 3:"Midfielder", 4:"Forward"}

# Get Data for each player

In [79]:
def check_num(f):
    try:
        return float(f)
    except:
        return f

async def get_player_data():
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()
        for player in players:
            keys = []
            vals = []
            for attr in dir(player):
                if "_" != attr[0]:
                    if attr in ["games_played","vapm"]:
                        val = await player.__getattribute__(attr)
                    else:
                        val = player.__getattribute__(attr)
                    val = check_num(val)
                    keys.append(attr)
                    vals.append(val)
            yield {k:v for k,v in zip(keys,vals)}


In [80]:
data = get_player_data()
i = 0
async for d in data:
    try:
        player_df.loc[i] = d.values()
    except:
        player_df = pd.DataFrame(columns=d.keys())
        player_df.loc[i] = d.values()
    finally:
        i+=1

In [81]:
player_df = player_df.convert_dtypes()

In [82]:
player_df.dtypes

assists                           Int64
bonus                             Int64
bps                               Int64
chance_of_playing_next_round      Int64
chance_of_playing_this_round      Int64
                                 ...   
value_form                      float64
value_season                    float64
vapm                            float64
web_name                         string
yellow_cards                      Int64
Length: 70, dtype: object

In [83]:
player_df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,vapm,web_name,yellow_cards
0,0,0,0,0,0,0,37605,<NA>,,0,...,0,3264,55,47258,3038,0.0,0.0,0.000000,Özil,0
1,0,0,0,0,0,0,39476,<NA>,,0,...,0,10195,18,17729,598,0.0,0.0,0.000000,Sokratis,0
2,0,0,51,50,100,0,41270,<NA>,,0,...,7,29500,2291,57672,10247,0.3,1.3,-0.109091,David Luiz,0
3,1,0,73,100,100,1,54694,<NA>,,-1,...,20,354384,5952,2509284,178237,0.2,1.7,0.113960,Aubameyang,1
4,0,0,0,100,100,0,58822,<NA>,,-1,...,0,2566,200,29736,937,0.0,0.0,0.000000,Cédric,0


In [88]:
for col in player_df:
    if len(player_df[col].unique()) == len(player_df):
        print(col)

code
creativity_rank
ict_index_rank
id
influence_rank
photo
threat_rank


In [89]:
player_df.drop(["code","id","photo"], axis=1, inplace=True)

In [90]:
today = date.today()
d = today.strftime("%d-%m-%Y")
player_df.to_csv("player_"+d+".csv")